Exploring the flow of influence random variables in a Bayesian network have on each other
----

In [1]:
from pomegranate import BayesianNetwork, DiscreteDistribution, ConditionalProbabilityTable, State

In [2]:
# helper function to pretty print observation results
def update(network, observations, variables=None):
    beliefs = network.forward_backward(observations)
    for state, dist in zip(network.states, beliefs):
        if variables is None or state.name in variables:
            fixed = {}
            for k, v in dist.parameters[0].items():
                fixed[k] = "{:.2}".format(v)
            print("{:<15}\t{}".format(state.name, fixed))

Setup the discrete probability distributions based on the Student Bayesian Network model

In [3]:
difficulty = DiscreteDistribution({'hard': 0.6, 'easy': 0.4})
intelligence = DiscreteDistribution({'high': 0.7, 'low': 0.3})
grade = ConditionalProbabilityTable([
        ['hard', 'high', 'A', 0.3],
        ['hard', 'high', 'B', 0.4],
        ['hard', 'high', 'C', 0.3],
        ['hard', 'low', 'A', 0.05],
        ['hard', 'low', 'B', 0.25],
        ['hard', 'low', 'C', 0.7],
        ['easy', 'high', 'A', 0.9],
        ['easy', 'high', 'B', 0.08],
        ['easy', 'high', 'C', 0.02],
        ['easy', 'low', 'A', 0.5],
        ['easy', 'low', 'B', 0.3],
        ['easy', 'low', 'C', 0.2]
    ], [difficulty, intelligence])
sat = ConditionalProbabilityTable([
        ['high', 'good', 0.95],
        ['high', 'bad', 0.05],
        ['low', 'good', 0.2],
        ['low', 'bad', 0.8],
    ], [intelligence])
letter = ConditionalProbabilityTable([
        ['A', 'unfavorable', 0.1],
        ['A', 'favorable', 0.9],
        ['B', 'unfavorable', 0.4],
        ['B', 'favorable', 0.6],
        ['C', 'unfavorable', 0.99],
        ['C', 'favorable', 0.01],
    ], [grade])

Create state objects (network nodes)

In [4]:
difficulty_state = State(difficulty, name='difficulty')
intelligence_state = State(intelligence, name='intelligence')
grade_state = State(grade, name='grade')
sat_state = State(sat, name='sat')
letter_state = State(letter, name='letter')

Construct the actual network by adding the nodes then creating the directed edges between them

In [5]:
student = BayesianNetwork("Student Network")
student.add_states([difficulty_state, intelligence_state, grade_state, sat_state, letter_state])
student.add_transition(difficulty_state, grade_state)
student.add_transition(intelligence_state, grade_state)
student.add_transition(intelligence_state, sat_state)
student.add_transition(grade_state, letter_state)
student.bake()

A child node is influenced by observing its parent

In [6]:
update(student, {'intelligence': 'high'}, 'sat')
update(student, {'intelligence': 'low'}, 'sat')

sat            	{'bad': '0.05', 'good': '0.95'}
sat            	{'bad': '0.8', 'good': '0.2'}


Likewise, a parent node is influenced by observing its child

In [7]:
update(student, {'sat': 'good'}, 'intelligence')
update(student, {'sat': 'bad'}, 'intelligence')

intelligence   	{'high': '0.92', 'low': '0.083'}
intelligence   	{'high': '0.13', 'low': '0.87'}


A grandchild can also be influenced by a grandparent and vice-versa

In [8]:
update(student, {'difficulty': 'hard'}, 'letter')
update(student, {'difficulty': 'easy'}, 'letter')

letter         	{'unfavorable': '0.58', 'favorable': '0.42'}
letter         	{'unfavorable': '0.21', 'favorable': '0.79'}


In [9]:
update(student, {'letter': 'unfavorable'}, 'difficulty')
update(student, {'letter': 'favorable'}, 'difficulty')

difficulty     	{'easy': '0.19', 'hard': '0.81'}
difficulty     	{'easy': '0.56', 'hard': '0.44'}


Siblings and cousins can also influence each other via a shared ancestor

In [10]:
update(student, {'letter': 'unfavorable'}, 'sat')
update(student, {'letter': 'favorable'}, 'sat')

sat            	{'bad': '0.38', 'good': '0.62'}
sat            	{'bad': '0.2', 'good': '0.8'}


In [11]:
update(student, {'sat': 'good'}, 'letter')
update(student, {'sat': 'bad'}, 'letter')

letter         	{'unfavorable': '0.37', 'favorable': '0.63'}
letter         	{'unfavorable': '0.59', 'favorable': '0.41'}


Independent parents with a shared child do not normally influence each other due to the v-structure

In [12]:
update(student, {'difficulty': 'hard'}, 'intelligence')
update(student, {'difficulty': 'easy'}, 'intelligence')

intelligence   	{'high': '0.7', 'low': '0.3'}
intelligence   	{'high': '0.7', 'low': '0.3'}


This can be overcome, however, by observing the shared child

In [13]:
update(student, {'difficulty': 'hard', 'grade': 'A'}, 'intelligence')
update(student, {'difficulty': 'easy', 'grade': 'A'}, 'intelligence')

intelligence   	{'high': '0.93', 'low': '0.067'}
intelligence   	{'high': '0.81', 'low': '0.19'}
